In [ ]:
from __future__ import print_function
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
np.random.seed(1337)  # for reproducibility
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding
from keras.layers import Dense, Dropout, Embedding, LSTM, Input, Bidirectional,GRU,SpatialDropout1D,SimpleRNN
from keras.datasets import imdb
from keras.utils.np_utils import to_categorical
from sklearn.metrics import (precision_score, recall_score,
                             f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn import metrics
from sklearn.preprocessing import Normalizer
import h5py
from keras import callbacks

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:

# Import twitter data
print(colored("Loading train and test data", "green"))

tr = pd.read_csv("/content/drive/My Drive/big.csv")
te = pd.read_csv("/content/drive/My Drive/twt_#ifc.csv")

tr.full_text=tr.full_text.astype(str)
te.full_text=te.full_text.astype(str)
print(colored("... \nData loaded", "green"))

Loading train and test data
... 
Data loaded


In [ ]:

# Tokenization
print(colored("Tokenizing and padding data", "green"))
tokenizer = Tokenizer(num_words = 2000, split = ' ')
tokenizer.fit_on_texts(tr['full_text'].astype(str).values)
train_tweets = tokenizer.texts_to_sequences(tr['full_text'].astype(str).values)
max_len = max([len(i) for i in train_tweets])
train_tweets = pad_sequences(train_tweets, maxlen = max_len)
test_tweets = tokenizer.texts_to_sequences(te['full_text'].astype(str).values)
test_tweets = pad_sequences(test_tweets, maxlen = max_len)
print(colored("Tokenizing and padding complete", "yellow"))

Tokenizing and padding data
Tokenizing and padding complete


In [ ]:
print(colored("Creating the Bidirectional SimpleRNN model", "yellow"))

modelbrnn = Sequential()
modelbrnn.add(Embedding(2000, 128, input_length = train_tweets.shape[1]))

modelbrnn.add(Bidirectional(SimpleRNN(128)))

modelbrnn.add(Dropout(0.1))

modelbrnn.add(Dense(3, activation = 'softmax'))
modelbrnn.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
modelbrnn.summary()

Creating the Bidirectional SimpleRNN model
Model: "sequential_38"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_34 (Embedding)     (None, 37, 128)           256000    
_________________________________________________________________
bidirectional_25 (Bidirectio (None, 256)               65792     
_________________________________________________________________
dropout_19 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_22 (Dense)             (None, 3)                 771       
Total params: 322,563
Trainable params: 322,563
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Training the model
print(colored("Training the Bidirectional SimpleRNN model", "green"))
history = modelbrnn.fit(train_tweets, pd.get_dummies(tr['sent_score']).values, epochs = 3, batch_size = 128)
print(colored(history, "green"))

Training the Bidirectional SimpleRNN model


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/3
20741/20741 [==============================] - 12s 569us/step - loss: 0.8776 - accuracy: 0.5681
Epoch 2/3
20741/20741 [==============================] - 11s 549us/step - loss: 0.6329 - accuracy: 0.7270
Epoch 3/3
20741/20741 [==============================] - 11s 545us/step - loss: 0.4359 - accuracy: 0.8240


In [ ]:
print(colored("Testing the Bidirectional SimpleRNNmodel", "green"))

loss, accuracy = modelbrnn.evaluate(train_tweets, pd.get_dummies(tr['sent_score']).values, batch_size = 128)
print(colored("Train accuracy: {}".format(accuracy),"yellow"))
x5=accuracy
score, accuracy = modelbrnn.evaluate(test_tweets, pd.get_dummies(te['sent_score']).values, batch_size = 128)
print(colored("Test accuracy: {}".format(accuracy),"yellow"))
y5=accuracy


Testing the Bidirectional SimpleRNNmodel
20741/20741 [==============================] - 3s 167us/step
Train accuracy: 0.9244973659515381
1749/1749 [==============================] - 0s 161us/step
Test accuracy: 0.8393367528915405


In [ ]:

# Building the model
print(colored("Creating the LSTM model", "yellow"))
modellstm = Sequential()
modellstm.add(Embedding(2000, 128, input_length = train_tweets.shape[1]))
modellstm.add(SpatialDropout1D(0.4))
modellstm.add(LSTM(256, dropout = 0.2,recurrent_dropout = 0.5,return_sequences = True))
modellstm.add(LSTM(256, dropout = 0.2,recurrent_dropout = 0.5,return_sequences = True))
modellstm.add(LSTM(256, dropout = 0.2,recurrent_dropout = 0.5,return_sequences = True))
modellstm.add(LSTM(256, dropout = 0.2,recurrent_dropout = 0.5,return_sequences = True))
modellstm.add(LSTM(256, dropout = 0.2,recurrent_dropout = 0.5,return_sequences = False))

modellstm.add(Dense(3, activation = 'softmax'))
modellstm.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
modellstm.summary()

Creating the LSTM model
Model: "sequential_39"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_35 (Embedding)     (None, 37, 128)           256000    
_________________________________________________________________
spatial_dropout1d_9 (Spatial (None, 37, 128)           0         
_________________________________________________________________
lstm_27 (LSTM)               (None, 37, 256)           394240    
_________________________________________________________________
lstm_28 (LSTM)               (None, 37, 256)           525312    
_________________________________________________________________
lstm_29 (LSTM)               (None, 37, 256)           525312    
_________________________________________________________________
lstm_30 (LSTM)               (None, 37, 256)           525312    
_________________________________________________________________
lstm_31 (LSTM)               

In [181]:
# Training the model
print(colored("Training the LSTM model", "green"))
history = modellstm.fit(train_tweets, pd.get_dummies(tr['sent_score']).values, epochs = 3, batch_size = 128)
print(colored(history, "green"))

Training the LSTM model


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/3
20741/20741 [==============================] - 357s 17ms/step - loss: 0.9264 - accuracy: 0.5343
Epoch 2/3
20741/20741 [==============================] - 355s 17ms/step - loss: 0.7882 - accuracy: 0.6215
Epoch 3/3
20741/20741 [==============================] - 350s 17ms/step - loss: 0.6686 - accuracy: 0.7083


In [182]:
print(colored("Testing the LSTM model", "green"))

loss, accuracy = modellstm.evaluate(train_tweets, pd.get_dummies(tr['sent_score']).values, batch_size = 128)
print(colored("Train accuracy: {}".format(accuracy),"yellow"))
x=accuracy
score, accuracy = modellstm.evaluate(test_tweets, pd.get_dummies(te['sent_score']).values, batch_size = 128)
print(colored("Test accuracy: {}".format(accuracy),"yellow"))
y=accuracy


Testing the LSTM model
20741/20741 [==============================] - 97s 5ms/step
Train accuracy: 0.7723349928855896
1749/1749 [==============================] - 8s 5ms/step
Test accuracy: 0.739279568195343


In [183]:
# Building the model
print(colored("Creating the GRU model", "yellow"))
modelgru = Sequential()
modelgru.add(Embedding(2000, 128, input_length = train_tweets.shape[1]))
modelgru.add(SpatialDropout1D(0.4))
modelgru.add(GRU(256, dropout = 0.2,recurrent_dropout = 0.5,return_sequences = True))
modelgru.add(GRU(256, dropout = 0.2,recurrent_dropout = 0.5,return_sequences = True))
modelgru.add(GRU(256, dropout = 0.2,recurrent_dropout = 0.5,return_sequences = True))
modelgru.add(GRU(256, dropout = 0.2,recurrent_dropout = 0.5,return_sequences = True))
modelgru.add(GRU(256, dropout = 0.2,recurrent_dropout = 0.5,return_sequences = False))

modelgru.add(Dense(3, activation = 'softmax'))
modelgru.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
modelgru.summary()

Creating the GRU model
Model: "sequential_40"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_36 (Embedding)     (None, 37, 128)           256000    
_________________________________________________________________
spatial_dropout1d_10 (Spatia (None, 37, 128)           0         
_________________________________________________________________
gru_9 (GRU)                  (None, 37, 256)           295680    
_________________________________________________________________
gru_10 (GRU)                 (None, 37, 256)           393984    
_________________________________________________________________
gru_11 (GRU)                 (None, 37, 256)           393984    
_________________________________________________________________
gru_12 (GRU)                 (None, 37, 256)           393984    
_________________________________________________________________
gru_13 (GRU)                 (

In [184]:
# Training the model
print(colored("Training the GRU model", "green"))
history = modelgru.fit(train_tweets, pd.get_dummies(tr['sent_score']).values, epochs = 3, batch_size = 128)
print(colored(history, "green"))

Training the GRU model


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/3
20741/20741 [==============================] - 365s 18ms/step - loss: 2636827.3462 - accuracy: 0.4218
Epoch 2/3
20741/20741 [==============================] - 360s 17ms/step - loss: 203727.9804 - accuracy: 0.4258
Epoch 3/3
20741/20741 [==============================] - 355s 17ms/step - loss: 64065.1102 - accuracy: 0.4202


In [185]:
print(colored("Testing the GRU model", "green"))

loss, accuracy = modelgru.evaluate(train_tweets, pd.get_dummies(tr['sent_score']).values, batch_size = 128)
print(colored("Train accuracy: {}".format(accuracy),"yellow"))
x1=accuracy
score, accuracy = modelgru.evaluate(test_tweets, pd.get_dummies(te['sent_score']).values, batch_size = 128)
print(colored("Test accuracy: {}".format(accuracy),"yellow"))
y1=accuracy

Testing the GRU model
20741/20741 [==============================] - 80s 4ms/step
Train accuracy: 0.5352200865745544
1749/1749 [==============================] - 7s 4ms/step
Test accuracy: 0.5397369861602783


In [186]:
# 1. define the network
print(colored("Creating the Bidirectional LSTM model", "yellow"))

modelblstm = Sequential()
modelblstm.add(Embedding(2000, 128, input_length = train_tweets.shape[1]))

modelblstm.add(Bidirectional(LSTM(128)))

modelblstm.add(Dropout(0.1))

modelblstm.add(Dense(3, activation = 'softmax'))
modelblstm.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
modelblstm.summary()

Creating the Bidirectional LSTM model
Model: "sequential_41"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_37 (Embedding)     (None, 37, 128)           256000    
_________________________________________________________________
bidirectional_26 (Bidirectio (None, 256)               263168    
_________________________________________________________________
dropout_20 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_25 (Dense)             (None, 3)                 771       
Total params: 519,939
Trainable params: 519,939
Non-trainable params: 0
_________________________________________________________________


In [187]:

# Training the model
print(colored("Training the Bidirectional LSTM model", "green"))
history = modelblstm.fit(train_tweets, pd.get_dummies(tr['sent_score']).values, epochs = 3, batch_size = 128)
print(colored(history, "green"))

Training the Bidirectional LSTM model


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/3
20741/20741 [==============================] - 53s 3ms/step - loss: 0.7831 - accuracy: 0.6305
Epoch 2/3
20741/20741 [==============================] - 52s 3ms/step - loss: 0.5693 - accuracy: 0.7512
Epoch 3/3
20741/20741 [==============================] - 51s 2ms/step - loss: 0.5204 - accuracy: 0.7814


In [188]:

print(colored("Testing the Bidirectional LSTM model", "green"))

loss, accuracy = modelblstm.evaluate(train_tweets, pd.get_dummies(tr['sent_score']).values, batch_size = 128)
print(colored("Train accuracy: {}".format(accuracy),"yellow"))
x2=accuracy
score, accuracy = modelblstm.evaluate(test_tweets, pd.get_dummies(te['sent_score']).values, batch_size = 128)
print(colored("Test accuracy: {}".format(accuracy),"yellow"))
y2=accuracy

Testing the Bidirectional LSTM model
20741/20741 [==============================] - 14s 653us/step
Train accuracy: 0.8148594498634338
1749/1749 [==============================] - 1s 653us/step
Test accuracy: 0.7821612358093262


In [189]:
# 1. define the network
print(colored("Creating the Bidirectional GRU model", "yellow"))

modelbgru = Sequential()
modelbgru.add(Embedding(2000, 128, input_length = train_tweets.shape[1]))

modelbgru.add(Bidirectional(GRU(128)))

modelbgru.add(Dropout(0.1))

modelbgru.add(Dense(3, activation = 'softmax'))
modelbgru.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
modelbgru.summary()

Creating the Bidirectional GRU model
Model: "sequential_42"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_38 (Embedding)     (None, 37, 128)           256000    
_________________________________________________________________
bidirectional_27 (Bidirectio (None, 256)               197376    
_________________________________________________________________
dropout_21 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_26 (Dense)             (None, 3)                 771       
Total params: 454,147
Trainable params: 454,147
Non-trainable params: 0
_________________________________________________________________


In [190]:
# Training the model
print(colored("Training the Bidirectional GRU model", "green"))
history = modelbgru.fit(train_tweets, pd.get_dummies(tr['sent_score']).values, epochs = 3, batch_size = 128)
print(colored(history, "green"))

Training the Bidirectional GRU model


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/3
20741/20741 [==============================] - 53s 3ms/step - loss: 0.7799 - accuracy: 0.6291
Epoch 2/3
20741/20741 [==============================] - 52s 2ms/step - loss: 0.5655 - accuracy: 0.7573
Epoch 3/3
20741/20741 [==============================] - 52s 2ms/step - loss: 0.5198 - accuracy: 0.7788


In [191]:

print(colored("Testing the Bidirectional GRU  model", "green"))

loss, accuracy = modelbgru.evaluate(train_tweets, pd.get_dummies(tr['sent_score']).values, batch_size = 128)
print(colored("Train accuracy: {}".format(accuracy),"yellow"))
x3=accuracy
score, accuracy = modelbgru.evaluate(test_tweets, pd.get_dummies(te['sent_score']).values, batch_size = 128)
print(colored("Test accuracy: {}".format(accuracy),"yellow"))
y3=accuracy

Testing the Bidirectional GRU  model
20741/20741 [==============================] - 12s 564us/step
Train accuracy: 0.816257655620575
1749/1749 [==============================] - 1s 559us/step
Test accuracy: 0.7827329635620117


In [192]:
# Building the model
print(colored("Creating the Vanilla RNN model", "yellow"))
modelvrnn = Sequential()
modelvrnn.add(Embedding(2000, 128, input_length = train_tweets.shape[1]))
modelvrnn.add(SpatialDropout1D(0.4))
modelvrnn.add(SimpleRNN(256,return_sequences = True))
modelvrnn.add(SimpleRNN(256,return_sequences = False))

modelvrnn.add(Dense(3, activation = 'softmax'))
modelvrnn.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
modelvrnn.summary()

Creating the Vanilla RNN model
Model: "sequential_43"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_39 (Embedding)     (None, 37, 128)           256000    
_________________________________________________________________
spatial_dropout1d_11 (Spatia (None, 37, 128)           0         
_________________________________________________________________
simple_rnn_5 (SimpleRNN)     (None, 37, 256)           98560     
_________________________________________________________________
simple_rnn_6 (SimpleRNN)     (None, 256)               131328    
_________________________________________________________________
dense_27 (Dense)             (None, 3)                 771       
Total params: 486,659
Trainable params: 486,659
Non-trainable params: 0
_________________________________________________________________


In [193]:
# Training the model
print(colored("Training the Vanilla RNN model", "green"))
history = modelvrnn.fit(train_tweets, pd.get_dummies(tr['sent_score']).values, epochs = 3, batch_size = 128)
print(colored(history, "green"))

Training the Vanilla RNN model


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/3
20741/20741 [==============================] - 35s 2ms/step - loss: 0.9458 - accuracy: 0.5186
Epoch 2/3
20741/20741 [==============================] - 34s 2ms/step - loss: 0.7946 - accuracy: 0.6194
Epoch 3/3
20741/20741 [==============================] - 34s 2ms/step - loss: 0.6243 - accuracy: 0.7333


In [194]:
print(colored("Testing the Vanilla RNN model", "green"))

loss, accuracy = modelvrnn.evaluate(train_tweets, pd.get_dummies(tr['sent_score']).values, batch_size = 128)
print(colored("Train accuracy: {}".format(accuracy),"yellow"))
x4=accuracy
score, accuracy = modelvrnn.evaluate(test_tweets, pd.get_dummies(te['sent_score']).values, batch_size = 128)
print(colored("Test accuracy: {}".format(accuracy),"yellow"))
y4=accuracy

Testing the Vanilla RNN model
20741/20741 [==============================] - 11s 548us/step
Train accuracy: 0.8334217071533203
1749/1749 [==============================] - 1s 552us/step
Test accuracy: 0.7873070240020752


In [195]:
from tabulate import tabulate


In [197]:
print(tabulate([
    ['LSTM',x,y], 
    ['GRU',x1,y1],
    ['Bi-LSTM',x2,y2],
    ['Bi-GRU',x3,y3],
    ['SimpleRNN/V-RNN',x4,y4],
    ['Bi-SimpleRNN',x5,y5]], 
    
    headers=['Model', 'Training Accuracy','Testing Accuracy']))

Model              Training Accuracy    Testing Accuracy
---------------  -------------------  ------------------
LSTM                        0.772335            0.73928
GRU                         0.53522             0.539737
Bi-LSTM                     0.814859            0.782161
Bi-GRU                      0.816258            0.782733
SimpleRNN/V-RNN             0.833422            0.787307
Bi-SimpleRNN                0.924497            0.839337
